Step1 : 데이터 수집하기
웹캠에서 얼굴 부분만 검출하여 사진 100장 찍어 폴더에 저장하기

1) cv2.CascadeClassifier() 객체 선언하기
2) cv2.VideoCapture(0) 객체 선언하기
3) 웹캠이 열려있으면
frame 을 읽어서
회색조로 변경
detectMultiScale() 사용하여 얼굴 감지하기
감지된 얼굴에 대해 사이즈  줄여 지정한 폴더에 구분자를 붙여 이미지로 저장하기(cv2.imwrite())
화면에는 카운트 숫자 표기하기
'q'로 종료하거나 count 가 100일때 종료하기

Step2 : 모델 학습하기
저장된 얼굴 이미지를 읽어서 모델 학습하기
1) 저장된 폴더에서 이미지를 읽어(cv2.imread() ) 회색조로 저장하기
2) 100장의 이미지를 리스트에 학습할 데이터(Training_Data) 와 Labels을 append 하여 각각 저장하기
3) 모델 생성하기 : model = cv2.face.LBPHFaceRecognizer_create()
4) 모델 학습하기 : model.train(Training_Data, Labels)
5) 훈련된 모델 파일로 저장하기 :model.write('face_model.yml')

Step3 : 얼굴 인식하기
분류기를 사용하여 웹캠에 비친 얼굴 중 학습한 사용자 인식하기
1) cv2.CascadeClassifier() 객체 선언하기
2) cv2.VideoCapture(0) 객체 선언하기
3) 웹캠이 열려있으면
frame 을 읽어서
회색조로 변경
detectMultiScale() 사용하여 얼굴 감지하기
얼굴이 검출된 경우
회색조로 변경하고
모델 예측하기 : model.predict()
예측 결과 신뢰도로가 75 이상인경우 화면에 "ACCESS GRANTED"
        아니면 "ACCESS DENIED"
'q'로 종료

In [1]:
import cv2              # OpenCV 라이브러리 가져오기
import numpy as np      # Numpy 라이브러리 가져오기
import sys
import os
import pandas as pd
from pathlib import Path

Training_Data = []
Labels = []

for i in range(100):        
    x = cv2.imread("./data_CCS/image"  + str(i) + ".jpg", cv2.IMREAD_GRAYSCALE)
    
    if x is not None:
        Training_Data.append(x)
        Labels.append(1)
    
for i in range(130):
    x = cv2.imread("./data_CJH/saved_image"  + format(i, '03') + ".jpg", cv2.IMREAD_GRAYSCALE)
    
    if x is not None:
        Training_Data.append(x)
        Labels.append(2)
    
for i in range(100):    
    x = cv2.imread("./data_JHW/"  + str(i) + ".jpg", cv2.IMREAD_GRAYSCALE)
    
    if x is not None:
        Training_Data.append(x)
        Labels.append(3)
    
for i in range(100):
    x = cv2.imread("./data_KJD/image_"  + str(i) + ".jpg", cv2.IMREAD_GRAYSCALE)
    
    if x is not None:
        Training_Data.append(x)
        Labels.append(4)        
    
for i in range(100):
    x = cv2.imread("./data_KOJ/img_"  + format(i, '03') + ".jpg", cv2.IMREAD_GRAYSCALE)
    
    if x is not None:
        Training_Data.append(x)
        Labels.append(5)

In [2]:
import matplotlib.pyplot as plt

def matdraw(img):
    plt.axis('off') # 창에있는 x축 y축 제거
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
    
# 다수 mat 그리기 함수
def matsdraw(imgs, row, col):
    for idx in range(len(imgs)):
        plt.subplot(row, col, idx + 1)
        plt.axis('off') # 창에있는 x축 y축 제거
        plt.imshow(cv2.cvtColor(imgs[idx], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
for i in range(10):
    matdraw(Training_Data[i * 50])

In [4]:
print(type(Training_Data))
print(type(Training_Data[i]))
print(type(Labels))
print(len(Labels))

unique, counts = np.unique(Labels, return_counts = True)
uniq_cnt_dict = dict(zip(unique, counts))

uniq_cnt_dict

<class 'list'>
<class 'numpy.ndarray'>
<class 'list'>
459


{1: 59, 2: 100, 3: 100, 4: 100, 5: 100}

In [5]:
model = cv2.face.LBPHFaceRecognizer_create()

model.train(Training_Data, np.array(Labels))
model.write('team_face_model.yml')

In [6]:
import cv2              # OpenCV 라이브러리 가져오기
import numpy as np      # Numpy 라이브러리 가져오기
import sys
import os
from pathlib import Path

# cv2.CascadeClassifier() 
# face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 전통적인 cv로 만든 classifier임
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) #' 첫 번째' 카메라(웹캠)로 VideoCapture 객체 생성  

model=cv2.face.LBPHFaceRecognizer_create();
model.read("team_face_model.yml")
confidence = 0

while(True):
    ret, frame = camera.read()                              # 프레임 단위로 캡처    
    frame = cv2.resize(frame, (800, 600))
    faces = face_classifier.detectMultiScale(frame, 1.3, 5)
    
    # 화면에 비친 인원 모두에게 사각형 표시
    # 접근 허가라고 출력
    if(len(faces) > 0): 
        for (x, y, w, h) in faces:
            crop = frame[y:y+h, x:x+w].copy()   
            crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            
            result = model.predict(crop)            
        
            confidence = int(100 * (1 - (result[1] / 300)))
            
            if (result[1] < 50):            
                if((result[0] == 1) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "Hi! CCS", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 100, 100),3)
                    
                elif((result[0] == 2) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "Hi! CJH", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 100, 100),3)
                    
                elif((result[0] == 3) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "Hi! JHW", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 100, 100),3)
                    
                elif((result[0] == 4) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "Hi! KJD", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 100, 100),2)
                    
                elif((result[0] == 5) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "Hi! KOJ", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 100, 100),3)
                else:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 100, 255), 5)
                    cv2.putText(frame, "Who...?", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (100, 100, 255),3)
            
            else:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 255, 255), 5)
                cv2.putText(frame, "Not sure", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (100, 255, 255),3)
            

    cv2.imshow('Press q to Exit',frame)              # 프레임 표시
    
    if (cv2.waitKey(1) & 0xFF == ord('q')):  # 스페이스바가 감지되면 중지
        break

camera.release()                           # 스페이스바가 감지된 후 창을 종료
cv2.destroyAllWindows()